In [1]:
import time
import pandas as pd
import requests
import json
import concurrent.futures
from decouple import config

# Disable warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
def get_jsonparsed_data(url):
    """
    Sends a GET request to FMP's Earning Surprise API and returns the resulting data in a dictionary
    """
    # sending get request and saving the response as response object
    response = requests.get(url=url)
    data = json.loads(response.text)
    return data

In [3]:
# Read in nasdaq data and extract symbols column
nasdaq_list = pd.read_csv("../data/nasdaq_list.csv")
# Pull API key from .env file
FMP_API_KEY = config("FMP_API_KEY")

nasdaq_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8202 entries, 0 to 8201
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      8202 non-null   object 
 1   Name        8202 non-null   object 
 2   Market Cap  7729 non-null   float64
 3   Country     7458 non-null   object 
 4   IPO Year    4820 non-null   float64
 5   Sector      6178 non-null   object 
 6   Industry    6195 non-null   object 
dtypes: float64(2), object(5)
memory usage: 448.7+ KB


In [4]:
# Filter missing out rows with missing data
# Filter reduces dataset from 8202 entries to 3156 entries
nasdaq_list_filtered = nasdaq_list.dropna()
nasdaq_list_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3156 entries, 0 to 8200
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      3156 non-null   object 
 1   Name        3156 non-null   object 
 2   Market Cap  3156 non-null   float64
 3   Country     3156 non-null   object 
 4   IPO Year    3156 non-null   float64
 5   Sector      3156 non-null   object 
 6   Industry    3156 non-null   object 
dtypes: float64(2), object(5)
memory usage: 197.2+ KB


In [5]:
symbols = nasdaq_list_filtered.Symbol

url_list = []
for idx, val in enumerate(symbols):
    url = "https://financialmodelingprep.com/api/v3/historical/earning_calendar/{}?limit=80&apikey={}".format(val, FMP_API_KEY)
    url_list.append(url)

In [6]:
# Call FMP API for each URL using Concurrent library
# Run takes 211 seconds ... be patient
with concurrent.futures.ThreadPoolExecutor() as executor:
    res = [executor.submit(get_jsonparsed_data, url) for url in url_list]
    concurrent.futures.wait(res)

pending: 0 jobs
threads: 1
pending: 0 jobs
threads: 2
pending: 0 jobs
threads: 3
pending: 0 jobs
threads: 4
pending: 0 jobs
threads: 5
pending: 0 jobs
threads: 6
pending: 0 jobs
threads: 7
pending: 0 jobs
threads: 8
pending: 0 jobs
threads: 9
pending: 0 jobs
threads: 10
pending: 0 jobs
threads: 11
pending: 0 jobs
threads: 12
pending: 0 jobs
threads: 13
pending: 0 jobs
threads: 14
pending: 0 jobs
threads: 15
pending: 0 jobs
threads: 16
pending: 1 jobs
threads: 16
pending: 2 jobs
threads: 16
pending: 3 jobs
threads: 16
pending: 4 jobs
threads: 16
pending: 5 jobs
threads: 16
pending: 6 jobs
threads: 16
pending: 7 jobs
threads: 16
pending: 8 jobs
threads: 16
pending: 9 jobs
threads: 16
pending: 10 jobs
threads: 16
pending: 11 jobs
threads: 16
pending: 12 jobs
threads: 16
pending: 13 jobs
threads: 16
pending: 14 jobs
threads: 16
pending: 15 jobs
threads: 16
pending: 16 jobs
threads: 16
pending: 17 jobs
threads: 16
pending: 18 jobs
threads: 16
pending: 19 jobs
threads: 16
pending: 20 jobs
th

In [12]:
res[90].result()

IndexError: list index out of range

In [ ]:
df = pd.DataFrame()
for x in range(len(symbols)):
    res_df = pd.DataFrame(res[x].result())
    df = pd.concat([df, res_df])

In [ ]:
print(df)

In [ ]:
df.info()

In [ ]:
df.to_csv("../data/test.csv")